In [ ]:
import os
lines = {}
for dirname, dirpaths, files in os.walk('.'):
    if "history_codes" in dirname: continue
    for file in files:
        if file.endswith('.py'):
            filepath = os.path.join(dirname, file)
            with open(filepath) as f:
                lines = {
                    **lines,
                    **{filepath: len(f.readlines())}
                }
[print(line) for line in lines.items()]
print(f"Total lines of code: {sum(lines.values())}")

In [ ]:
import zipfile, os
from tqdm.auto import tqdm
from best_results import paths, paths_flunet

best_result_paths = {'dataforgood': paths, 'flunet': paths_flunet}
country_names = {
    **{'EN': "England", 'FR': "France", 'IT': 'Italy', 'ES': 'Spain', 'NZ': 'NewZealand', 'JP': 'Japan'},
    **{'h1n1': 'h1n1', 'h3n2': 'h3n2', 'BY': 'BY', 'BV': 'BV'}
}

total = sum([1 for dataset, paths_dataset in best_result_paths.items() for observed_ratio in paths_dataset for key in paths_dataset[observed_ratio].index])


qbar = tqdm(total=total)

with zipfile.ZipFile("results.zip", "w") as f:
    for dataset, paths_dataset in best_result_paths.items():
        for observed_ratio in paths_dataset:
            for y, country_code, model_name in paths_dataset[observed_ratio].index:
                old_model_path = paths_dataset[observed_ratio].loc[y, country_code, model_name].path
                old_args_path = os.path.join(os.path.dirname(old_model_path), country_names[country_code], 'args.txt')
                new_model_path = os.path.join('{}_{}_y{}_{}_{}.pth'.format(dataset, observed_ratio, y, country_code, model_name))
                new_args_path = os.path.join('{}_{}_y{}_{}_{}_args.txt'.format(dataset, observed_ratio, y, country_code, model_name))
                f.write(old_model_path, arcname=new_model_path)
                f.write(old_args_path, arcname=new_args_path)
                qbar.update()

  0%|          | 0/160 [00:00<?, ?it/s]